In [5]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.datasets import make_classification
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
x, y = make_classification(n_samples=2000, n_classes=2, weights=[0.7, 0.3], random_state=42)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

**Basic Stacking**

In [3]:
base_models = [
    ('dt', DecisionTreeClassifier(max_depth=5, random_state=42)),
    ('svm', SVC(probability=True, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

meta_model = LogisticRegression(random_state=42)

# Create stacking classifier
stacking = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5, # Cross-validation for base predictions
    stack_method='predict_proba', # Use probabilities
    n_jobs=-1
)

stacking.fit(x_train, y_train)
y_pred = stacking.predict(x_test)
print(f"Stacking Accuracy: {accuracy_score(y_test, y_pred):.3f}")

Stacking Accuracy: 0.943


*Detailed Stacking Process*
- Step 1: Train Base Models

In [6]:
# Train base models and get out-of-fold predictions
base_predictions = {}

for name, model in base_models:
    pred = cross_val_predict(model, x_train, y_train, cv=5, method='predict_proba')
    base_predictions[name] = pred
    print(f"{name} trained")

x_meta = np.hstack([base_predictions[name] for name, _ in base_models])
print(f"Meta features shape: {x_meta.shape}")

dt trained
svm trained
knn trained
Meta features shape: (1600, 6)


- Step 2: Train Meta-Model

In [8]:
# Train meta-model on base predictions
meta_model.fit(x_meta, y_train)

meta_score = meta_model.score(x_meta, y_train)
print(f"Meta-model training score: {meta_score:.3f}")

Meta-model training score: 0.905


- Step 3: Make Predictions

In [ ]:
test_base_predictions = {}

for name, model in base_models:
    model.fit(x_train, y_train)
    pred = model.predict_proba(x_test)
    test_base_predictions[name] = pred
    
x_test_meta = np.hstack([test_base_predictions[name] for name, _ in base_models])

final_predictions = meta_model.predict(x_test_meta)